In [36]:
!pip install optuna

You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [37]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [38]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [39]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [41]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-11-26 00:48:10,098] A new study created in memory with name: no-name-8af5575d-7e3e-44ff-b377-156f95c90572
[I 2025-11-26 00:48:12,270] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 95, 'max_depth': 16}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-26 00:48:13,834] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 107, 'max_depth': 6}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-26 00:48:15,186] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 132, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-26 00:48:16,375] Trial 3 finished with value: 0.756052141527002 and parameters: {'n_estimators': 185, 'max_depth': 3}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-26 00:48:18,198] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 184, 'max_depth': 17}. Best is trial 4 with value: 0.770949720

In [42]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7858472998137803
Best hyperparameters: {'n_estimators': 121, 'max_depth': 15}


In [43]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


## Samplers in Optuna

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [45]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-11-26 00:48:47,126] A new study created in memory with name: no-name-1c3503fd-9fc5-4410-8abd-5e6ef9ae7650
[I 2025-11-26 00:48:47,377] Trial 0 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 55, 'max_depth': 20}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:48:47,638] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 58, 'max_depth': 19}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:48:48,273] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 151, 'max_depth': 16}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:48:48,788] Trial 3 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 124, 'max_depth': 9}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:48:49,556] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 194, 'max_depth': 8}. Best is trial 0 with value: 0.77653631

In [46]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 124, 'max_depth': 18}


In [47]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [48]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [49]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-11-26 00:49:12,005] A new study created in memory with name: no-name-db931b5d-0de8-40b9-91d6-2c3222bd5156
[I 2025-11-26 00:49:12,382] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-26 00:49:13,006] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-26 00:49:13,218] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-11-26 00:49:13,646] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-11-26 00:49:14,087] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [50]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [51]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [52]:
!pip install plotly


You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [53]:
!pip install nbformat


You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [54]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [55]:
# 1. Optimization History
plot_optimization_history(study).show()

In [56]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [57]:
# 3. Slice Plot
plot_slice(study).show()

In [58]:
# 4. Contour Plot
plot_contour(study).show()

In [59]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [60]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [61]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [62]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-11-26 00:49:24,412] A new study created in memory with name: no-name-de59e3cb-40dc-4ed2-8492-0324b5e279ac
[I 2025-11-26 00:49:25,239] Trial 0 finished with value: 0.7765363128491621 and parameters: {'classifier': 'RandomForest', 'n_estimators': 297, 'max_depth': 4, 'min_samples_split': 3, 'min_samples_leaf': 6, 'bootstrap': False}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:49:25,962] Trial 1 finished with value: 0.7746741154562384 and parameters: {'classifier': 'RandomForest', 'n_estimators': 257, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 10, 'bootstrap': False}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:49:29,375] Trial 2 finished with value: 0.7430167597765363 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 268, 'learning_rate': 0.024115891059079762, 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-11-26 00:49:29,401] Trial 

KeyboardInterrupt: 

In [ ]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.13215967753542082, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.757914,2025-11-26 00:45:15.253688,2025-11-26 00:45:15.287905,0 days 00:00:00.034217,5.251047,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.767225,2025-11-26 00:45:15.288961,2025-11-26 00:45:16.472662,0 days 00:00:01.183701,NaN,True,RandomForest,NaN,NaN,NaN,8.0,3.0,2.0,258.0,COMPLETE
2,2,0.696462,2025-11-26 00:45:16.473671,2025-11-26 00:45:16.490700,0 days 00:00:00.017029,17.704160,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.767225,2025-11-26 00:45:16.493456,2025-11-26 00:45:17.099423,0 days 00:00:00.605967,NaN,False,RandomForest,NaN,NaN,NaN,15.0,5.0,8.0,171.0,COMPLETE
4,4,0.763501,2025-11-26 00:45:17.100401,2025-11-26 00:45:17.986438,0 days 00:00:00.886037,NaN,True,RandomForest,NaN,NaN,NaN,7.0,8.0,4.0,249.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.716946,2025-11-26 00:45:40.796394,2025-11-26 00:45:40.814459,0 days 00:00:00.018065,0.159934,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.789572,2025-11-26 00:45:40.815253,2025-11-26 00:45:40.832316,0 days 00:00:00.017063,0.115015,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-11-26 00:45:40.833829,2025-11-26 00:45:40.851471,0 days 00:00:00.017642,0.241173,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.746741,2025-11-26 00:45:40.853459,2025-11-26 00:45:41.509510,0 days 00:00:00.656051,NaN,True,RandomForest,NaN,NaN,NaN,18.0,2.0,9.0,164.0,COMPLETE


In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

SVM                 76
RandomForest        15
GradientBoosting     9
Name: params_classifier, dtype: int64

In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.754811
RandomForest        0.765363
SVM                 0.773890
Name: value, dtype: float64

In [ ]:
# 1. Optimization History
plot_optimization_history(study).show()

In [ ]:
# 3. Slice Plot
plot_slice(study).show()

In [ ]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [63]:
! pip install optuna-integration[xgboost]

You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [64]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-11-26 00:49:48,784] A new study created in memory with name: no-name-7ef902a7-8733-4202-a46a-d981191fc8ed


[0]	train-mlogloss:0.80516	eval-mlogloss:0.78968
[1]	train-mlogloss:0.68027	eval-mlogloss:0.67669
[2]	train-mlogloss:0.54062	eval-mlogloss:0.52507
[3]	train-mlogloss:0.42779	eval-mlogloss:0.40772
[4]	train-mlogloss:0.35213	eval-mlogloss:0.32613
[5]	train-mlogloss:0.29195	eval-mlogloss:0.26066
[6]	train-mlogloss:0.25110	eval-mlogloss:0.21327
[7]	train-mlogloss:0.21764	eval-mlogloss:0.17709
[8]	train-mlogloss:0.18887	eval-mlogloss:0.14667
[9]	train-mlogloss:0.16960	eval-mlogloss:0.12628
[10]	train-mlogloss:0.15467	eval-mlogloss:0.11318
[11]	train-mlogloss:0.13982	eval-mlogloss:0.10085
[12]	train-mlogloss:0.13043	eval-mlogloss:0.09106
[13]	train-mlogloss:0.12141	eval-mlogloss:0.08439
[14]	train-mlogloss:0.11409	eval-mlogloss:0.07802
[15]	train-mlogloss:0.10950	eval-mlogloss:0.07312
[16]	train-mlogloss:0.10544	eval-mlogloss:0.06860
[17]	train-mlogloss:0.10103	eval-mlogloss:0.06512
[18]	train-mlogloss:0.09861	eval-mlogloss:0.06454
[19]	train-mlogloss:0.09584	eval-mlogloss:0.06258
[20]	train

[I 2025-11-26 00:49:49,266] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.3848845976627773, 'alpha': 0.7569540565885057, 'eta': 0.2643360888436703, 'gamma': 1.154649468243334e-07, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.772446726234115, 'colsample_bytree': 0.6771634402696408}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.78743	eval-mlogloss:0.78505
[1]	train-mlogloss:0.58861	eval-mlogloss:0.56975
[2]	train-mlogloss:0.44366	eval-mlogloss:0.40956
[3]	train-mlogloss:0.34660	eval-mlogloss:0.30399
[4]	train-mlogloss:0.27284	eval-mlogloss:0.24134
[5]	train-mlogloss:0.22142	eval-mlogloss:0.18285
[6]	train-mlogloss:0.18590	eval-mlogloss:0.13890
[7]	train-mlogloss:0.15936	eval-mlogloss:0.10911
[8]	train-mlogloss:0.14327	eval-mlogloss:0.08773
[9]	train-mlogloss:0.13110	eval-mlogloss:0.07450
[10]	train-mlogloss:0.11168	eval-mlogloss:0.06095
[11]	train-mlogloss:0.10861	eval-mlogloss:0.05685
[12]	train-mlogloss:0.10240	eval-mlogloss:0.05430
[13]	train-mlogloss:0.09781	eval-mlogloss:0.05031
[14]	train-mlogloss:0.09617	eval-mlogloss:0.04909
[15]	train-mlogloss:0.09533	eval-mlogloss:0.04730
[16]	train-mlogloss:0.09412	eval-mlogloss:0.04495
[17]	train-mlogloss:0.09435	eval-mlogloss:0.04416
[18]	train-mlogloss:0.09448	eval-mlogloss:0.04554
[19]	train-mlogloss:0.09247	eval-mlogloss:0.04427
[20]	train

[I 2025-11-26 00:49:49,450] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.0534849248397912e-08, 'alpha': 0.0021074449221124823, 'eta': 0.27238824545374213, 'gamma': 0.006939306184453169, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.5216117889563529, 'colsample_bytree': 0.7938244117832596}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.80401	eval-mlogloss:0.80630
[1]	train-mlogloss:0.60709	eval-mlogloss:0.59290


[I 2025-11-26 00:49:53,776] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.77419	eval-mlogloss:0.76971
[1]	train-mlogloss:0.58429	eval-mlogloss:0.55796
[2]	train-mlogloss:0.44099	eval-mlogloss:0.40199


[I 2025-11-26 00:49:53,813] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 0.3848845976627773, 'alpha': 0.7569540565885057, 'eta': 0.2643360888436703, 'gamma': 1.154649468243334e-07, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.772446726234115, 'colsample_bytree': 0.6771634402696408}
Best accuracy: 1.0


In [65]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()